In [ ]:
import os
import glob

In [ ]:
def save_fsleyes_vis(Anat,Effect,Stat,Out,Threshold):
    #6 would be above max/min for all effect sizes (all very similar)
    #8.5 would be above max/min for all stat sizes (sst and nback very similar, mid lower)
    ! fsleyes render --neuroOrientation --outfile $Out -sz  500 500 \
       --selectedOverlay 1 --scene ortho --worldLoc 0 0 0 \
       --zcentre  0  0 --showLocation no \
       --layout horizontal --hidex --hidey --hideCursor \
       --showColourBar --colourBarLocation right --colourBarLabelSide top-left --colourBarSize 85.0 --labelSize 14 \
       $Anat --overlayType volume  \
       $Effect --name "" --overlayType volume --modulateImage $Stat --cmap red-yellow --negativeCmap blue-lightblue --useNegativeCmap --displayRange 0 6 --clippingRange 0 6 --modulateRange 0.0 8.5 --modulateAlpha \
       $Stat --overlayType mask --alpha 100.0 --threshold -$Threshold $Threshold --outline --maskColour 0 0 0
    return

In [ ]:
def get_files(group,ses,task,contrast):
    
    if len(glob.glob(f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_effect_size.nii.gz')) == 0:
        print('No effect size maps, so cannot generate fsleyes output for: '+' '.join([group,ses,task,contrast]))
        return (None, None, None, None, None)

    
    #get anatfile, statmap and effectmap
    anat_file='../templates/tpl-MNI152NLin6Asym_res-01_desc-brain_T1w.nii.gz'
    effect_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_effect_size.nii.gz'
    stat_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_z_score.nii.gz'

    #get threshold
    threshold_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_FDR_threshold.txt'
    with open(threshold_file) as f:
        threshold = float(f.readline().strip('\n'))
    rounded_threshold = round(threshold,2)

    #get participant count
    participants_file = f'../../../derivatives/task_analysis_volume/group_level/group-{group}/ses-{ses}/task-{task}/group-{group}_ses-{ses}_task-{task}_rec-unco_contrast-{contrast}_participants.txt'
    with open(participants_file) as f:
        part_count = len(f.readlines())

    #create paths to output dir if not exist
    derivatives_path = '../../../derivatives'
    nilearn_output_path = os.path.join(derivatives_path, 'task_analysis_volume','visualization','raw_figures')
    if not os.path.isdir(nilearn_output_path):
        os.makedirs (nilearn_output_path)

    #make outfile path
    out_file = f'../../../derivatives/task_analysis_volume/visualization/raw_figures/group-{group}_ses-{ses}_task-{task}_contrast-{contrast}_threshold-{rounded_threshold}_n-{part_count}_display-axial.png'

    if threshold > 100:
        print('Threshold is infinity, so cannot generate fsleyes output for: '+' '.join([group,ses,task,contrast]))
        return (None, None, None, None, None)

    return (anat_file,effect_file,stat_file,out_file,threshold)

In [ ]:
task='sst'
sessions = ['baseline','1year']
groups = ['HC','MM']


if task == 'mid':
    contrasts=['HiWin+LoWin-HiNoWin-LoNoWin','HiLoss+LoLoss-AvoidHiLoss-AvoidLoLoss']

elif task == 'sst':
    contrasts=['SuccStop-Go','UnsuccStop-Go','UnsuccStop-SuccStop']
    
elif task == 'nback':
    contrasts=['twoback-zeroback']
    
    
for ses in sessions:
    for group in groups:        
        for contrast in contrasts:
            
            anat_file,effect_file,stat_file,out_file,threshold = get_files(group,ses,task,contrast)
            
            #only call if input found 
            if effect_file:
                save_fsleyes_vis(anat_file,effect_file,stat_file,out_file,threshold)